In [1]:
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
import torch
from torch import nn
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
import os
from PIL import Image
from transformers import SegformerForSemanticSegmentation, SegformerImageProcessor ,SegformerFeatureExtractor,AutoModelForSemanticSegmentation
import pandas as pd
import cv2
import numpy as np
from torch.utils.data import DataLoader
import albumentations as aug
from torchinfo import summary
from matplotlib import pyplot as plt
from torchvision.transforms import ColorJitter


/home/cplus/projects/m.tarek_master/m.tarek/m.tarek/lib/python3.8/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2024-02-20 14:49:55.731680: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-20 14:49:55.752901: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-20 14:49:56.122751: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class CustomDataset(Dataset):

    def __init__(self, root_dir, feature_extractor, transforms=None, train=True):
        super(CustomDataset,self).__init__()
        self.root_dir = root_dir
        self.feature_extractor = feature_extractor
        self.train = train
        self.transforms = transforms
        self.img_dir = os.path.join(self.root_dir, "images")
        self.ann_dir = os.path.join(self.root_dir, "masks")
        
        image_file_names = []
        for root, dirs, files in os.walk(self.img_dir):
            image_file_names.extend(files)
        self.images = sorted(image_file_names)
        annotation_file_names = []
        for root, dirs, files in os.walk(self.ann_dir):
            annotation_file_names.extend(files)
        self.annotations = sorted(annotation_file_names)

    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        image = cv2.imread(os.path.join(self.img_dir, self.images[idx]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        segmentation_map = cv2.imread(os.path.join(self.ann_dir, self.annotations[idx]))
        segmentation_map = cv2.cvtColor(segmentation_map, cv2.COLOR_BGR2GRAY)
        
        if self.transforms is not None:
            augmented = self.transforms(image=image, mask=segmentation_map)
            jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1) 


            encoded_inputs = self.feature_extractor(jitter(Image.fromarray(augmented['image'])), augmented['mask'], return_tensors="pt")
        else:
            encoded_inputs = self.feature_extractor(image, segmentation_map, return_tensors="pt")

        for k,v in encoded_inputs.items():
            encoded_inputs[k].squeeze_()

        return encoded_inputs

In [3]:
transform = aug.Compose([
    aug.Flip(p=0.5)
],is_check_shapes=False)

In [6]:
train_dir =r"/home/cplus/projects/m.tarek_master/m.tarek/graval detection project/datasets/under_water_masks_dataset/train"
valid_dir=r"/home/cplus/projects/m.tarek_master/m.tarek/graval detection project/datasets/under_water_masks_dataset/val"
test_dir=r"/home/cplus/projects/m.tarek_master/m.tarek/graval detection project/datasets/under_water_masks_dataset/test"
feature_extractor = SegformerImageProcessor.from_pretrained ("nvidia/mit-b5")
train_dataset = CustomDataset(root_dir=train_dir, feature_extractor=feature_extractor, transforms=transform)
valid_dataset = CustomDataset(root_dir=valid_dir, feature_extractor=feature_extractor, transforms=None, train=False)
test_dataset = CustomDataset(root_dir=test_dir, feature_extractor=feature_extractor, transforms=None, train=False)

In [7]:
encoded_inputs = train_dataset[0]
img=encoded_inputs["pixel_values"].detach().cpu().numpy().reshape((512,512,3))
mask=encoded_inputs["labels"].detach().cpu().numpy()

In [8]:
print(img)

[[[-1.0390445  -1.004795   -1.0219197 ]
  [-1.004795   -1.0219197  -0.9876702 ]
  [-1.004795   -1.004795   -0.9876702 ]
  ...
  [-1.0219197  -1.0219197  -1.0219197 ]
  [-0.9876702  -0.9876702  -1.0219197 ]
  [-1.004795   -0.9705454  -0.9705454 ]]

 [[-1.1246684  -1.0561693  -1.0219197 ]
  [-1.0219197  -1.004795   -1.004795  ]
  [-0.9876702  -0.9705454  -0.9362959 ]
  ...
  [-1.0390445  -1.073294   -1.0390445 ]
  [-1.0390445  -1.0219197  -1.004795  ]
  [-1.0219197  -1.004795   -0.9876702 ]]

 [[-1.0904187  -1.0561693  -1.073294  ]
  [-1.0390445  -1.004795   -1.004795  ]
  [-1.0219197  -1.0561693  -1.0219197 ]
  ...
  [-1.0561693  -1.0561693  -1.0561693 ]
  [-1.073294   -1.073294   -1.073294  ]
  [-1.073294   -1.1075436  -1.1246684 ]]

 ...

 [[ 0.68793046  0.6705013   0.6356429 ]
  [ 0.6705013   0.65307206  0.65307206]
  [ 0.65307206  0.68793046  0.7227889 ]
  ...
  [-0.18352933 -0.11381256 -0.11381256]
  [-0.13124175 -0.18352933 -0.14867094]
  [-0.20095852 -0.14867094 -0.18352933]]

 [

In [9]:
for i in mask:
    print(i)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [10]:
classes = ["stone"]
print(classes)
id2label = {0:classes[0]}
print(id2label)
label2id = {v: k for k, v in id2label.items()}
print(label2id)

['stone']
{0: 'stone'}
{'stone': 0}


In [11]:
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/mit-b5", ignore_mismatched_sizes=True,
                                                         num_labels=len(classes),id2label=id2label,label2id=label2id,
                                                         reshape_last_stage=True)

/home/cplus/projects/m.tarek_master/m.tarek/m.tarek/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b5 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj

In [12]:
for para in model.parameters():
    para.requires_grad=True

In [13]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=4, shuffle=True)
valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=4,shuffle=False)
test_dataloader=DataLoader(dataset=test_dataset,batch_size=4,shuffle=False)

In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Model Initialized!")
print(model.device)


Model Initialized!
cpu


In [15]:
summary(model=model)

Layer (type:depth-idx)                                                      Param #
SegformerForSemanticSegmentation                                            --
├─SegformerModel: 1-1                                                       --
│    └─SegformerEncoder: 2-1                                                --
│    │    └─ModuleList: 3-1                                                 1,929,408
│    │    └─ModuleList: 3-2                                                 79,511,552
│    │    └─ModuleList: 3-3                                                 2,048
├─SegformerDecodeHead: 1-2                                                  --
│    └─ModuleList: 2-2                                                      --
│    │    └─SegformerMLP: 3-4                                               49,920
│    │    └─SegformerMLP: 3-5                                               99,072
│    │    └─SegformerMLP: 3-6                                               246,528
│    │    └─Segf

In [16]:
for epoch in range(1, 11):  # loop over the dataset multiple times
    print("Epoch:", epoch)
    pbar = tqdm(train_dataloader)
    accuracies = []
    losses = []
    val_accuracies = []
    val_losses = []
    model.train()
    for idx, batch in enumerate(pbar):
        # get the inputs;
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(pixel_values=pixel_values, labels=labels)

        # evaluate
        upsampled_logits = nn.functional.interpolate(outputs.logits, size=labels.shape[-2:], mode="bilinear", align_corners=False)
        predicted = upsampled_logits.argmax(dim=1)

        mask = (labels != 255) # we don't include the background class in the accuracy calculation
        pred_labels = predicted[mask].detach().cpu().numpy()
        true_labels = labels[mask].detach().cpu().numpy()
        accuracy = accuracy_score(pred_labels, true_labels)
        loss = outputs.loss
        accuracies.append(accuracy)
        losses.append(loss.item())
        pbar.set_postfix({'Batch': idx, 'Pixel-wise accuracy': sum(accuracies)/len(accuracies), 'Loss': sum(losses)/len(losses)})

        # backward + optimize
        loss.backward()
        optimizer.step()
        #plt.imshow((pixel_values[0].detach().cpu().numpy()).reshape(512, 512,3))
        #plt.imshow(pred_labels)
        #plt.imshow(true_labels)
        plt.show()
    else:
        model.eval()
        with torch.no_grad():
            for idx, batch in enumerate(valid_dataloader):
                pixel_values = batch["pixel_values"].to(device)
                labels = batch["labels"].to(device)

                outputs = model(pixel_values=pixel_values, labels=labels)
                upsampled_logits = nn.functional.interpolate(outputs.logits, size=labels.shape[-2:], mode="bilinear", align_corners=False)
                predicted = upsampled_logits.argmax(dim=1)

                mask = (labels != 255) # we don't include the background class in the accuracy calculation
                pred_labels = predicted[mask].detach().cpu().numpy()
                true_labels = labels[mask].detach().cpu().numpy()
                accuracy = accuracy_score(pred_labels, true_labels)
                val_loss = outputs.loss
                val_accuracies.append(accuracy)
                val_losses.append(val_loss.item())

    print(f"Train Pixel-wise accuracy: {sum(accuracies)/len(accuracies)}\
         Train Loss: {sum(losses)/len(losses)}\
         Val Pixel-wise accuracy: {sum(val_accuracies)/len(val_accuracies)}\
         Val Loss: {sum(val_losses)/len(val_losses)}")

Epoch: 1


  0%|          | 0/145 [00:00<?, ?it/s]

IndexError: list index out of range

In [ ]:
%load_ext tensorboard
%tensorboard --logdir segformer_pure_pytorch_log/